In [1]:
import pandas as pd
import numpy as np
import datetime as dt

# Module for visualization

In [2]:
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots




## RE is the module for regex . 

In [4]:
import re

## Convert source to dataframe

In [5]:
df = pd.read_json('code_source.py')

Convert string date to datetime 

In [7]:
df['transaction_date'] = pd.to_datetime(df['transaction_date'] )

In [8]:
df['birthdate'] = pd.to_datetime(df['birthdate'] )

check latest date and farthest date of the dataframe

In [9]:
df['month'] = df['transaction_date'].dt.month_name()

In [10]:
df['month_num'] = df['transaction_date'].dt.month

In [11]:
df['transaction_date'].max()

Timestamp('2022-06-30 00:00:00')

In [12]:
df['transaction_date'].min()

Timestamp('2022-01-01 00:00:00')

In [ ]:
# list_[1]

In [ ]:
# inactive_list = df.groupby(['month_num']).username.apply(lambda x: ','.join(x)).fillna('').reset_index().rename({'username': 'previous_list'}, axis=1)

## Transform to Granularity

Assignning the split key for the column transaction_items. 

The current format is 'HealthyKid 3+,Gummy Vitamins,(x3);HealthyKid 3+,Yummy Vegetables,(x2)'
1st step is to split by ";"
2nd step is to split by ","
and get the value with (x3) for the quantity of each item

In [13]:
df['transaction_items'][2]

'HealthyKid 3+,Gummy Vitamins,(x3);HealthyKid 3+,Yummy Vegetables,(x2)'

In [15]:
df = df.assign(var=df['transaction_items'].str.split(';'))

# Convert to Row

In [16]:
df = df.explode('var')

# Repeating the same process above 

In [17]:
df['number_of_purchases'] = df['var'].str.split('(',expand=True)[1].str.replace('x','').str.replace(')','').astype(int)

C:\Users\kubales\AppData\Local\Temp\ipykernel_25352\3064442970.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['number_of_purchases'] = df['var'].str.split('(',expand=True)[1].str.replace('x','').str.replace(')','').astype(int)


In [18]:
df = df.assign(var=df['var'].str.split(','))

After assigning a split key , df.explode converts the assigned key to rows 

In [19]:
df = df.explode('var')

### Remove the frame with value "\\(x" in rows. it should be in new column

In [20]:
df = df[~df['var'].str.contains(f"\\(x")]

In [21]:
df['previous_month_num'] = df['month_num']-1

# rename assign var column to product name

In [22]:
df.rename(columns ={'var': 'product_name'},inplace=True)

In [23]:
df

,address,birthdate,mail,name,sex,username,transaction_items,transaction_value,transaction_date,month,month_num,product_name,number_of_purchases,previous_month_num
0,USNS Wagner\nFPO AE 00917,1926-03-11,watsonbrenda@gmail.com,Jennifer Campbell,F,veronicawells,"Exotic Extras,Beef Chicharon,(x4)",5196,2022-01-12,January,1,Exotic Extras,4,0
0,USNS Wagner\nFPO AE 00917,1926-03-11,watsonbrenda@gmail.com,Jennifer Campbell,F,veronicawells,"Exotic Extras,Beef Chicharon,(x4)",5196,2022-01-12,January,1,Beef Chicharon,4,0
1,USCGC Benson\nFPO AP 48794,2016-07-08,moorelisa@gmail.com,Melissa Kim,F,cgilbert,"Exotic Extras,Beef Chicharon,(x3);HealthyKid 3...",12056,2022-01-12,January,1,Exotic Extras,3,0
1,USCGC Benson\nFPO AP 48794,2016-07-08,moorelisa@gmail.com,Melissa Kim,F,cgilbert,"Exotic Extras,Beef Chicharon,(x3);HealthyKid 3...",12056,2022-01-12,January,1,Beef Chicharon,3,0
1,USCGC Benson\nFPO AP 48794,2016-07-08,moorelisa@gmail.com,Melissa Kim,F,cgilbert,"Exotic Extras,Beef Chicharon,(x3);HealthyKid 3...",12056,2022-01-12,January,1,HealthyKid 3+,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83035,"PSC 1089, Box 3720\nAPO AE 08973",2012-04-30,xjones@hotmail.com,Stephanie Russell,F,cmcdowell,"HealthyKid 3+,Nutrional Milk,(x1);Candy City,G...",2639,2022-06-03,June,6,Nutrional Milk,1,5
83035,"PSC 1089, Box 3720\nAPO AE 08973",2012-04-30,xjones@hotmail.com,Stephanie Russell,F,cmcdowell,"HealthyKid 3+,Nutrional Milk,(x1);Candy City,G...",2639,2022-06-03,June,6,Candy City,3,5
83035,"PSC 1089, Box 3720\nAPO AE 08973",2012-04-30,xjones@hotmail.com,Stephanie Russell,F,cmcdowell,"HealthyKid 3+,Nutrional Milk,(x1);Candy City,G...",2639,2022-06-03,June,6,Gummy Worms,3,5
83035,"PSC 1089, Box 3720\nAPO AE 08973",2012-04-30,xjones@hotmail.com,Stephanie Russell,F,cmcdowell,"HealthyKid 3+,Nutrional Milk,(x1);Candy City,G...",2639,2022-06-03,June,6,Candy City,1,5


### Number of Purchases by Product

 We used the group by to get the summarize number of transactions per month.  Group by in python is like group by in sql

In [ ]:
df_by_product = df.groupby(['month_num','month','product_name'])['number_of_purchases'].sum().reset_index().sort_values(['month_num','number_of_purchases'],axis=0,ascending= False)

Another group by to get the overall number of purchase by month 

In [ ]:
print(df_by_product)

In [ ]:
monthly_count_purchases = df_by_product.groupby(['month_num'])['number_of_purchases'].sum().reset_index()

In [ ]:
monthly_count_purchases

For the transaction value we have to drop the duplicates by transaction date and username, Since we converted the current data to rows. It creates duplicates in trasaction value. The granular Table we made is only applicable to the number of purchases . 

In [ ]:
monthly_numbers = df.drop_duplicates(subset = ['transaction_date','username'])

Group by month to get the transaction value by month. 

In [ ]:
monthly_numbers = monthly_numbers.groupby(['month','month_num'])['transaction_value'].sum().reset_index().sort_values(['month_num','transaction_value'],ascending=False)

In [ ]:
monthly_numbers = monthly_numbers.merge( monthly_count_purchases, how='left', on='month_num')

 Converted the data type of transaction value to float to show decimals

In [ ]:
monthly_numbers['transaction_value'] = monthly_numbers['transaction_value'].astype(float)

# Monthly Numbers

In [ ]:
monthly_numbers

## Monthly Repeater

1st We group by the data by month and  convert all the username to a single row by by applying lambda x.join()
2nd The shift(1) , it shifts the value of the month by 1 . 

In [26]:
previous_month_list = df.groupby(['month_num']).username.apply(lambda x: ','.join(x)).shift(1).fillna('').reset_index().rename({'username': 'previous_list'}, axis=1)


In [28]:
previous_month_list

,month_num,previous_list
0,1,
1,2,"veronicawells,veronicawells,cgilbert,cgilbert,..."
2,3,"veronicawells,veronicawells,veronicawells,vero..."
3,4,"veronicawells,veronicawells,cgilbert,cgilbert,..."
4,5,"cgilbert,cgilbert,cgilbert,cgilbert,qhernandez..."
5,6,"cgilbert,cgilbert,cgilbert,cgilbert,cgilbert,c..."


 We join the above dataframe to our main raw dataframe using pandas merge.
 And apply lambda to determine if the current username exist in previous_list column 

In [ ]:
repeater = pd.merge(df, previous_month_list, how='left', on='month_num')
repeater['repeaters'] =repeater.apply(lambda row: row['username'] in row['previous_list'], axis=1)


In [ ]:
repeater = repeater.drop_duplicates(subset = ['month','username'])

We applied np.select with the condition of if the column 'repeaters' is true value should be 1


In [ ]:
repeater['repeater_count'] = np.select([repeater['repeaters'] == True],[1],default=0)

Finally with the group by to get the monthly numbers by month

In [ ]:
repeater = repeater.groupby(['month_num'])['repeater_count'].sum().reset_index()

In [ ]:
repeater

## Monthly Active numbers

drop the duplicates by month and username since we only need to know if they exists in a current month

In [30]:
df_active_engaged = df.drop_duplicates(subset = ['month','username'])

Make a new column count with value of 1 

In [31]:
df_active_engaged['count'] = 1

First is get the unique values of  month_num column and apply it in loop.
Each value represents the month number of the month 
1 for January 2 for Februany and so on..... 
We used the logic of <= month_num to determine wheter if they exists in current month and previous month

Then we group by by username inside the loop to get if they exists in the current month. 
Then we asign x for column[month_num]

then we append it to the blank list we created. which is frame = []


In [35]:
frame = []
for x in df_active_engaged['month_num'].unique():
    df_ = df_active_engaged[df_active_engaged['month_num']<= x]
    df_ = df_.groupby(['username']).sum().reset_index()
    df_['month_num'] = x
#     df_ = df[['month_num','username','count']]
    frame.append(df_)
                       
    

We convert the blank frame which has already the data of the loop we created. Using pd.concat it convert the list to dataframe

In [36]:
df_active_engaged = pd.concat(frame)

In [37]:
df_active_engaged = df_active_engaged[['month_num','username','count']]

np.select is applied to determine whether they exists in the current month and the last previous month.
To check if they are active in everymonth, the column count should be equal to the month_num 

In [ ]:
df_active_engaged['engaged'] = np.select([(df_active_engaged['month_num'] - df_active_engaged['count']== 0) & (df_active_engaged['month_num'] != 1)],[1],default=0)

In [ ]:
df_active_engaged = df_active_engaged.groupby(['month_num'])['engaged'].sum().reset_index()

In [ ]:
df_active_engaged

## Inactive List

Drop the duplicate by month_num and username

In [39]:
df_inactive = df.drop_duplicates(subset = ['month_num','username'])

In [40]:
df_inactive = df_inactive[['month_num','username']]



We applied same logic from the active number list. 

Get the unique value of month_num column.

Apply it to loop.
x = month_num

First we assign
    df_a as active in the current month. We sliced the frame by the current month_num.
    df_i as the inactive list , we sliced the frame by the month_num < x which is the current month_num
    To avoid duplicate we drop duplicate by username
    Then we check if the username  from the df_i does not exists in df_a(active from prevous months) using "isin" . the operator "~" returns the opposite value of the logic we applied. 
    Then we do the group by
    and comes up with the iactive count by month

In [41]:
inactive_frame = []
for x in df_inactive['month_num'].unique():
    
    df_a = df_inactive[df_inactive['month_num'] == x]
    df_i = df_inactive[df_inactive['month_num']< x]
    df_i = df_i.drop_duplicates(subset = 'username')
    df_i = df_i[~df_i['username'].isin(df_a['username'])]
    df_i = df_i['username'].count()
    df_a = df_a.groupby(['month_num']).count().reset_index()
    df_a['inactive'] = df_i
    df_a['month_num_'] = x

    
#     df_ = df[['month_num','username','count']]
    inactive_frame.append(df_a)
                       

In [42]:
inactive_frame = pd.concat(inactive_frame)

In [43]:
inactive_frame = inactive_frame[['month_num_','inactive','username']]

In [44]:
inactive_frame.rename(columns ={'month_num_': 'month_num','username':'monthly_customer_count'},inplace=True)

In [45]:
inactive_frame

,month_num,inactive,monthly_customer_count
0,1,0,6583
0,2,1416,6630
0,3,1728,6658
0,4,1912,6568
0,5,1915,6587
0,6,1840,6665


# Summary

Merge all the frame we created earlier

In [ ]:
summary = monthly_numbers.merge(repeater,on='month_num',how='left')

In [ ]:
summary = summary.merge(df_active_engaged,on='month_num',how='left')

In [ ]:
summary = summary.merge(inactive_frame,on='month_num',how='left')

In [ ]:
summary = summary.sort_values(['month_num'],ascending=True)

slice the frame and only get the column we needed

In [ ]:
summary = summary[['month', 'transaction_value', 'number_of_purchases',

                   'repeater_count', 'engaged', 'inactive', 'monthly_customer_count']]

In [ ]:
rowEvenColor = 'lightgrey'
rowOddColor = 'white'


fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[[{"type": "table"}],
           [{"type": "scatter"}],
           [{"type": "scatter"}]]
)

fig = go.Figure(data=[go.Table(

    header=dict(values=['Month','Total Amount','# of Purchases','Repeater','Engaged','Inactive','# of Customers'],
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[summary.month, summary.transaction_value, summary.number_of_purchases, summary.repeater_count,
                      summary.engaged,summary.inactive,summary.monthly_customer_count,],
              #  fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*10],
               fill_color='lavender',
               align='left',
              height = 60))
],
              )





fig.update_layout(
    title_text = 'Digi Foods Monthly Summary Analysis',
    height = 800,
    margin = {'t':75, 'l':50},
    yaxis = {'domain': [0, .45]},
    xaxis2 = {'anchor': 'y2'},
    yaxis2 = {'domain': [.6, 1], 'anchor': 'x2', 'title': 'Goals'}
)


fig.show()

In [ ]:
fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[[{"type": "table"}],
           [{"type": "scatter"}],
           [{"type": "scatter"}]]
)


fig.add_trace(go.Bar(x=summary.month, y=summary.repeater_count, xaxis='x2', yaxis='y2',
                marker=dict(color='#0099ff'),
                name='Previous<br>Month<br>#'))

fig.add_trace(go.Bar(x=summary.month, y=summary.inactive, xaxis='x2', yaxis='y2',
                marker=dict(color='#40ffff'),
                name='Inactive<br>#'))

fig.add_trace(go.Bar(x=summary.month, y=summary.engaged, xaxis='x2', yaxis='y2',
                marker=dict(color='#0030ff'),
                name='Active<br>All<br>Months'))
fig.add_trace(go.Bar(x=summary.month, y=summary.monthly_customer_count, xaxis='x2', yaxis='y2',
                marker=dict(color='#003fff'),
                name='Current<br>Active<br>Customers'))


fig.update_layout(
    title_text = 'Digi Foods Customer Analysis',
    height = 800,
    margin = {'t':100, 'l':50},
    yaxis = {'domain': [0, .45]},
    xaxis2 = {'anchor': 'y2'},
    yaxis2 = {'domain': [.6, 1], 'anchor': 'x2', 'title': 'Number Of Customers'}
)


fig.show()

In [ ]:
summary

### Slice the frame with the columns we only want to show in the visualization table

In [ ]:
df_by_product = df_by_product[['month','product_name','number_of_purchases']]

In [ ]:
df_by_product.rename(columns={'month':'Month','product_name':"Product Name","number_of_purchases":"Number Of Purchases"},inplace=True)

In [ ]:
fig = ff.create_table(df_by_product)
fig.show()